# Tarea 2: Red Neuronal Perceptrón Multicapa con TensorFlow V2

## Cargamos las librerias

In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [2]:
from tensorflow.keras.datasets.fashion_mnist import load_data
fashion_mnist = load_data()

## Extraemos los datos

In [3]:
(x_train, y_train), (x_test, y_test)=fashion_mnist

## Union de Datos

El dataset Fashion_Mnist cuenta con 60,000 imagenes de entrenamiento y 10,000 imagenes de prueba; Para los fines de esta tarea, se hara una división del dataset con una relación 30-70, es decir, %30 del dataset será destinado a test y %70 a entrenamiento.

In [4]:
X=np.concatenate((x_train,x_test))
Y=np.concatenate((y_train,y_test))

In [6]:
X=X.reshape(len(X),28*28).astype('float32')

## Codificación One Hot Encoding 

Las clases se representan con etiquetas numericas en el rango de $0,\cdots,9$:

<div align="center">

| Identificador 	| Clase 	|
|---	|---	|
| 0 	| T-shirt/top 	|
| 1 	| Trouser 	|
| 2 	| Pullover 	|
| 3 	| Dress 	|
| 4 	| Coat 	|
| 5 	| Sandal 	|
| 6 	| Shirt 	|
| 7 	| Sneaker 	|
| 8 	| Bag 	|
| 9 	| Ankle Boot 	|

</div>

Debido al identificador puede (o no) realizarse una codificación del tipo One Hot. Esta codificación permite que el modelo neuronal interprete la distancia entre identificadores de la misma forma, es decir, en un mal aprendizaje puede asociar la clase "Trouser" con la clase "Pullover" por que los identificadores son cercanos entre si, de forma comtraria es posible que interprete una relación (erronea) entre los identificadores mas separadas, lo cual no es relevante.

A fin de observar el efecto de la codificación se realizaran dos modelos: uno con codificación ONE HOT y otro sin ella.


In [ ]:
ONE_HOT_ENC = OneHotEncoder(sparse=False)
Y_OHE = Y.reshape(len(Y), 1)
Y_OHE = ONE_HOT_ENC.fit_transform(Y_OHE)

In [15]:
print("Las dimensiones del vector de etiquetas son {}".format(Y.shape))
print("Los primeros 10 elementos son: \n {}".format(Y[0:10]))

Las dimensiones del vector de etiquetas son (70000, 10)
Los primeros 10 elementos son 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


## Arquitectura de el modelo

Se aprovecha el paradigma de la programación orientada a objetos de Python para desarrollar una instancia que permita la declaración de la arquitectura del modelo MLP.

In [ ]:
class DNN_Model():
    def __init__(self,lenght_input):
        self.Layer_1_Weights=tf.Variable(np.random.rand(lenght_input,800))
